In [13]:
# Import necessary library files
import time
import threading
import cv2
import numpy as np
import tensorflow as tf

print("Tensorflow version", tf.__version__)

Tensorflow version 2.3.1


In [14]:
# Common params
alert_status = 'OFF'
intruder_status = False
file_id = 0

# Camera params
frame_rate = 24
frame_width = 1280
frame_height = 720
frame_size = (frame_width, frame_height)

In [15]:
# Load the Object Detection model
model = tf.saved_model.load('efficientdet', tags=None, options=None)

In [20]:
# To look for intruders in an image
def detect(image, confidence):
    global intruder_status
    found = False
    # Resize image
    img = cv2.resize(image, (640, 640))
    image_np = np.expand_dims(img, axis=0)
    # Run object detection
    detector_output = model(image_np)
    classes = detector_output["detection_classes"][0].numpy()[:20]
    scores = detector_output["detection_scores"][0].numpy()[:20]
    for i in range(20):
        # If a person is detected with a minimum confidence, start recording
        if classes[i] == 1.0 and scores[i] > confidence:
            intruder_status = True
            found = True
            print("Intruder!")
            break
    if found == False:
        intruder_status = False

In [21]:
# To monitor for and record intruders
def camera():
    global file_id
    frame_count = 0
    record_switch = False

    print("Starting camera...")
    cap = cv2.VideoCapture(0)
    result = cv2.VideoWriter('recordings/recording_' + str(file_id) + '.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, frame_size)

    while True:
        r, img = cap.read()
        if intruder_status == True and record_switch == False:
            # Start recording
            result = cv2.VideoWriter('recordings/recording_' + str(file_id) + '.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, frame_size)
            record_switch = True
            file_id += 1
            result.write(img)
        elif intruder_status == True and record_switch == True:
            # Recording
            result.write(img)
        elif intruder_status == False and record_switch == True:
            # Stop Recording
            record_switch = False
            result.release()
        frame_count += 1
        # Run detection every 1 second
        if frame_count % frame_rate == 0:
            frame_count = 0
            threading.Thread(target=detect, args=(img,0.6)).start()

        cv2.imshow("Live Preview", img)
        if alert_status == 'OFF':
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Demo: Monitoring for 1 minute
alert_status = 'ON'
threading.Thread(target=camera, args=()).start()
time.sleep(60)
alert_status = 'OFF'

Starting camera...
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
Intruder!
